In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
            .master("local[*]") \
            .appName("test") \
            .getOrCreate()

25/03/08 07:07:34 WARN Utils: Your hostname, SRCIND-21BQ9G3 resolves to a loopback address: 127.0.1.1; using 172.26.144.30 instead (on interface eth0)
25/03/08 07:07:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/08 07:07:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/08 07:07:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df_green = spark.read.parquet("data/pq/green/*/*")

In [4]:
df_green.registerTempTable("green")

/root/spark/spark-3.5.5-bin-hadoop3/python/pyspark/sql/dataframe.py:329: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [34]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) as hour, 
    PULocationID as zone,

    sum(total_amount) as revenue_monthly_total_amount,
    COUNT(1) as number_of_records
    FROM 
        green
    WHERE
        lpep_pickup_datetime > '2020-01-01 00:00:00'
    GROUP BY 
        1,2
""")

In [35]:
df_green_revenue \
    .repartition(20) \
    .write.parquet("data/report/revenue/green", mode="overwrite")

25/03/08 07:22:58 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [36]:
df_yellow = spark.read.parquet("data/pq/yellow/*/*")

In [37]:
df_yellow.registerTempTable("yellow")

In [38]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) as hour, 
    PULocationID as zone,

    sum(total_amount) as revenue_monthly_total_amount,
    COUNT(1) as number_of_records
    FROM 
        yellow
    WHERE
        tpep_pickup_datetime > '2020-01-01 00:00:00'
    GROUP BY 
        1,2
""")

In [39]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet("data/report/revenue/yellow", mode="overwrite")

25/03/08 07:23:09 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
25/03/08 07:23:10 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

## Joins

In [47]:
df_green_revenue = spark.read.parquet("data/report/revenue/green/")
df_yellow_revenue = spark.read.parquet("data/report/revenue/yellow/")

In [48]:
df_green_revenue_temp = df_green_revenue \
                            .withColumnRenamed('revenue_monthly_total_amount', 'green_amount') \
                            .withColumnRenamed('number_of_records', 'green_number_records') 

df_yellow_revenue_temp = df_yellow_revenue \
                            .withColumnRenamed('revenue_monthly_total_amount', 'yellow_amount') \
                            .withColumnRenamed('number_of_records', 'yellow_number_records') 

In [49]:
df_join = df_green_revenue_temp.join(df_yellow_revenue_temp, on=['hour', 'zone'], how='outer')

In [50]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

25/03/08 07:29:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                

In [51]:
df_join = spark.read.parquet('data/report/revenue/total/')

In [54]:
df_zones = spark.read.parquet('zones/')
# df_zones.show()

In [59]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [64]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/zones')

25/03/08 07:34:50 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
                                                                                